<div style="text-align: center; padding: 20px;">
<h1 align="center" style="font-size: 28px; color:rgb(156, 90, 255, 1); width: 100%;">━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━<br>تمرین ۵ - بازیابی هوشمند اطلاعات<br>━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━</h1>
<p style="color: #666; font-size: 16px;">طراح تمرین:  محمد جواد رنجبر</p>

<div style="border: 2px dashed rgba(156, 90, 255, 1); border-radius: 8px; padding: 20px; margin: 20px auto; max-width: 500px; text-align: right;">
<p style="color: rgba(156, 90, 255, 1); font-size: 18px; margin-bottom: 15px;"> مشخصات دانشجو:</p>
<p style="color: #666; margin: 5px;">نام و نام خانوادگی: روژین پناوp>
<p style="color: #666; margin: 5px;">شماره دانشجویی: 220701046p>
<p style="color: #666; margin: 5px;">تاریخ ارسال: .........................</p>
</div>
</div>

<div dir="rtl" style="text-align: justify;  padding: 25px; background-color:rgba(60, 4, 82, 0.21); border-radius: 12px; border: 2px solidrgb(2, 34, 22);"
<div style="line-height: 2.0; font-size: 17px;">
<div style="padding-right:40px">
</div>
<br>
<div style="padding-right:100px">
 ساختار تمرین:

<b>بخش ۱: مجموعه داده </b>

<b>بخش ۲: بازیابی با استفاده از مدل‌های پیش آموزش دیده</b>

<b>بخش ۳: بازیابی با استفاده از مدل‌های تنظیم دقیق شده</b>

<b>بخش ۴: آموزش مدل‌ها با استفاده از تنظیم دقیق کارآمد پارامتری</b>

<b>بخش ۵: ارزیابی جامع مدل‌‌ها</b>

<b>بخش ۶: رنگ کردن HYBRID</b>

<b>بخش ۷: پیاده سازی یک RERANKER مبتنی بر LLM با استفاده از GPT-5.1-MINI API</b>

</div>



</div>
</div>

<div style="text-align:center; direction:rtl; font-family:'Vazir','Tahoma',sans-serif; margin:40px 0; box-sizing: border-box;">
  <h1 style="
    font-size:26px;
    font-weight:700;
    padding:25px 20px;
    color:rgba(60, 4, 82, 0.81);
    border-radius:12px;
    background:linear-gradient(135deg,#f5f7ff,#ecebff);
    line-height:1.8;
  ">
   بخش ۱: مجموعه داده 
  </h1>
</div>


<div dir="rtl">

## آماده‌سازی مجموعه‌داده BEIR/nfcorpus برای بازیابی اطلاعات پزشکی

در این بخش، فرآیند بارگذاری، تحلیل ساختاری و آماری مجموعه‌داده **BEIR/nfcorpus** با رویکرد مهندسی بازیابی اطلاعات تشریح می‌شود. تمرکز اصلی بر آماده‌سازی داده (Data Engineering) به‌عنوان پیش‌نیاز حیاتی برای پیاده‌سازی مدل‌های Sparse و Neural Ranking است. تمامی تحلیل‌ها **صرفاً بر اساس خروجی‌های اجرایی ارائه‌شده** انجام شده و هیچ‌گونه داده یا الگوی فرضی وارد تحلیل نشده است.


## وابستگی‌ها و محیط اجرایی

### شرح پیاده‌سازی و منطق
در نخستین گام، کتابخانه‌های پایه شامل `numpy`، `pandas` و `plotly` بارگذاری شده‌اند. این انتخاب نشان‌دهنده تمرکز بر:
- پردازش برداری داده‌ها (`numpy`)
- تحلیل و تبدیل ساختاری داده‌های جدولی (`pandas`)
- مصورسازی آماری و تحلیلی (`plotly`)

خروجی موفق بارگذاری هر سه کتابخانه، تضمین‌کننده آمادگی محیط برای پردازش داده‌های حجیم متنی است.

### تحلیل نتایج تجربی
خروجی جدول زیر نشان می‌دهد که تمامی وابستگی‌ها بدون خطا بارگذاری شده‌اند و محیط اجرایی پایدار است:

<div dir="ltr">

| library | status |
|-------|--------|
| numpy | loaded |
| pandas | loaded |
| plotly | loaded |

</div>

این مرحله اگرچه ساده به نظر می‌رسد، اما از منظر مهندسی سیستم، پیش‌شرط تکرارپذیری (Reproducibility) آزمایش‌ها محسوب می‌شود.

</div>


In [89]:
import numpy as _np
import pandas as _pd
import plotly.graph_objects as _go
from pathlib import Path
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import CountVectorizer
import re
from collections import Counter
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import matplotlib.pyplot as plt


In [2]:
_pd.DataFrame(
    {
        "library": ["numpy", "pandas", "plotly"],
        "status": ["loaded", "loaded", "loaded"]
    }
)


,library,status
0,numpy,loaded
1,pandas,loaded
2,plotly,loaded


<div dir="rtl">

## بارگذاری و تحلیل پیکره اسنادی (Corpus)

### شرح پیاده‌سازی و منطق
کلاس `CorpusStore` با هدف انتزاع فرآیند بارگذاری پیکره طراحی شده است. در این کلاس:
- داده‌ها از فایل `jsonl` خوانده می‌شوند.
- در موارد فقدان عنوان، مقدار خالی جایگزین می‌شود.
- یک فیلد ترکیبی `full_text` ایجاد می‌شود که حاصل الحاق عنوان و متن است.

این تصمیم معماری، مستقیماً با چالش **ناهمگونی ساختار اسناد پزشکی** مرتبط است؛ زیرا برخی اسناد فاقد عنوان هستند و حذف آن‌ها باعث از دست رفتن اطلاعات بالقوه می‌شود.

### تحلیل نتایج تجربی
بر اساس خروجی اجرا:

<div dir="ltr">

| documents | avg_length |
|-----------|------------|
| 3633 | 1590.783925 |

</div>

تحلیل:
- تعداد اسناد (۳۶۳۳) با توصیف «تقریباً ۳۶۰۰ سند پزشکی» کاملاً هم‌خوان است.
- میانگین طول اسناد (~۱۵۹۱ کاراکتر) نشان‌دهنده **اسناد بلند و غنی از اطلاعات** است که در بازیابی اطلاعات پزشکی امری رایج است.
- این طول بالا، چالش sparsity را برای روش‌های کلاسیک کاهش می‌دهد اما در عین حال، هزینه محاسباتی مدل‌های عصبی را افزایش می‌دهد.

</div>


In [3]:
class CorpusStore:
    def __init__(self, corpus_path):
        self.corpus_path = corpus_path
        self.df = self._load()

    def _load(self):
        records = _pd.read_json(self.corpus_path, lines=True)
        records["title"] = records["title"].fillna("")
        records["full_text"] = records["title"] + " " + records["text"]
        return records[["_id", "title", "text", "full_text"]]

corpus_store = CorpusStore("nfcorpus/corpus.jsonl")
corpus_store.df.head(5)



,_id,title,text,full_text
0,MED-10,Statin Use and Breast Cancer Survival: A Natio...,"Recent studies have suggested that statins, an...",Statin Use and Breast Cancer Survival: A Natio...
1,MED-14,Statin use after diagnosis of breast cancer an...,BACKGROUND: Preclinical studies have shown tha...,Statin use after diagnosis of breast cancer an...
2,MED-118,Alkylphenols in human milk and their relations...,The aims of this study were to determine the c...,Alkylphenols in human milk and their relations...
3,MED-301,Methylmercury: A Potential Environmental Risk ...,Epilepsy or seizure disorder is one of the mos...,Methylmercury: A Potential Environmental Risk ...
4,MED-306,Sensitivity of Continuous Performance Test (CP...,Hit Reaction Time latencies (HRT) in the Conti...,Sensitivity of Continuous Performance Test (CP...


In [4]:
_pd.DataFrame(
    {
        "documents": [corpus_store.df.shape[0]],
        "avg_length": [corpus_store.df["full_text"].str.len().mean()]
    }
)


,documents,avg_length
0,3633,1590.783925


<div dir="rtl">

## بارگذاری و تحلیل پرس‌وجوها (Queries)

### شرح پیاده‌سازی و منطق
کلاس `QueryStore` مسئول بارگذاری پرس‌وجوها از فایل `queries.jsonl` است. پرس‌وجوها به‌صورت طبیعی (Natural Language) و کوتاه هستند که این ویژگی، شکاف معنایی (Semantic Gap) میان پرس‌وجو و اسناد را تشدید می‌کند.

### تحلیل نتایج تجربی
خروجی آماری پرس‌وجوها به‌صورت زیر است:

<div dir="ltr">

| queries | avg_query_length |
|---------|------------------|
| 3237 | 21.933272 |

</div>

تحلیل:
- تعداد پرس‌وجوها ۳۲۳۷ است که نسبت به عدد ۳۲۳ ذکرشده در توصیف اولیه، نشان می‌دهد نسخه مورد استفاده شامل **پرس‌وجوهای توسعه‌یافته‌تر** است.
- میانگین طول پرس‌وجو (~۲۲ کاراکتر) نشان‌دهنده پرس‌وجوهای بسیار کوتاه است.
- این عدم تقارن طولی میان پرس‌وجو و سند، یکی از دلایل اصلی نیاز به روش‌های Neural Ranking و Query Expansion در مراحل بعدی است.

</div>


In [5]:
class QueryStore:
    def __init__(self, query_path):
        self.query_path = query_path
        self.df = self._load()

    def _load(self):
        return _pd.read_json(self.query_path, lines=True)

query_store = QueryStore("nfcorpus/queries.jsonl")
query_store.df.sample(5)


,_id,text,metadata
1254,PLAIN-1724,nitrite,{'url': 'http://nutritionfacts.org/topics/nitr...
182,PLAIN-245,How Much Soy Is Too Much?,{'url': 'http://nutritionfacts.org/2013/02/19/...
1485,PLAIN-2032,saccharin,{'url': 'http://nutritionfacts.org/topics/sacc...
1007,PLAIN-1388,hyperpigmentation,{'url': 'http://nutritionfacts.org/topics/hype...
3066,PLAIN-1690,National Academy of Sciences,{'url': 'http://nutritionfacts.org/topics/nati...


In [6]:
_pd.DataFrame(
    {
        "queries": [query_store.df.shape[0]],
        "avg_query_length": [query_store.df["text"].str.len().mean()]
    }
)


,queries,avg_query_length
0,3237,21.933272


<div dir="rtl">

## دادگان ارتباط (Qrels) و تحلیل آماری Relevance

### شرح پیاده‌سازی و منطق
کلاس `RelevanceJudgments` برای پردازش فایل‌های qrels طراحی شده است. نکات کلیدی:
- ستون `score` ابتدا به‌صورت رشته خوانده شده و سپس به عدد صحیح تبدیل شده است.
- این تبدیل برای جلوگیری از خطاهای نوع داده و حفظ **graded relevance** ضروری است.

سه نمونه مستقل برای `train`، `dev` و `test` ساخته شده‌اند تا از **نشت داده (Data Leakage)** جلوگیری شود.

### تحلیل عمیق نتایج تجربی
خلاصه آماری تعداد اسناد مرتبط به‌ازای هر پرس‌وجو در هر بخش به شکل زیر است:

<div dir="ltr">

| split | count | mean | std | min | 25% | 50% | 75% | max |
|------|-------|------|-----|-----|-----|-----|-----|-----|
| train | 2590 | 42.693050 | 100.192802 | 1 | 6 | 18 | 41 | 1363 |
| dev | 324 | 35.138889 | 69.469925 | 1 | 6 | 18.5 | 39 | 919 |
| test | 323 | 38.185759 | 71.813225 | 1 | 5 | 16 | 39.5 | 475 |

تحلیل:
- میانگین اسناد مرتبط در هر پرس‌وجو در مجموعه آزمون ≈ **38.18** است که دقیقاً با مقدار اعلام‌شده (38.2) هم‌خوانی دارد.
- انحراف معیار بالا (بیش از ۷۰ در dev و test) نشان‌دهنده توزیع بسیار نامتوازن relevance است.
- وجود مقادیر حداکثری بسیار بزرگ (مثلاً 1363 در train) بیانگر آن است که برخی پرس‌وجوها دارای پوشش معنایی بسیار گسترده هستند.
- این ویژگی، استفاده از معیارهایی مانند **nDCG** را نسبت به Precision ساده ضروری می‌سازد.

</div>


In [7]:
class RelevanceJudgments:
    def __init__(self, qrels_path):
        self.df = _pd.read_csv(
            qrels_path,
            sep="\t",
            names=["query_id", "corpus_id", "score"],
            dtype={"query_id": str, "corpus_id": str, "score": str}
        )
        self.df["score"] = _pd.to_numeric(self.df["score"], errors="coerce").fillna(0).astype(int)

    def relevance_stats(self):
        return (
            self.df[self.df["score"] > 0]
            .groupby("query_id", as_index=False)
            .agg(relevant_docs=("corpus_id", "count"))
        )


In [ ]:
qrels_train = RelevanceJudgments("nfcorpus/qrels/train.tsv")
qrels_dev = RelevanceJudgments("nfcorpus/qrels/dev.tsv")
qrels_test = RelevanceJudgments("nfcorpus/qrels/test.tsv")

train_stats = qrels_train.relevance_stats().assign(split="train")
dev_stats = qrels_dev.relevance_stats().assign(split="dev")
test_stats = qrels_test.relevance_stats().assign(split="test")

all_stats = _pd.concat([train_stats, dev_stats, test_stats], axis=0)


In [9]:
all_stats.groupby("split")["relevant_docs"].describe()


,count,mean,std,min,25%,50%,75%,max
split,,,,,,,,
dev,324.0,35.138889,69.469925,1.0,6.0,18.5,39.0,919.0
test,323.0,38.185759,71.813225,1.0,5.0,16.0,39.5,475.0
train,2590.0,42.693050,100.192802,1.0,6.0,18.0,41.0,1363.0


<div dir="rtl">

## هم‌ترازی داده‌ها (Alignment Analysis)

### شرح پیاده‌سازی و منطق
در این بخش، هم‌پوشانی شناسه‌های موجود در qrels با داده‌های واقعی corpus و queries بررسی شده است. این تحلیل برای اطمینان از صحت اتصال سه جزء اصلی سیستم IR انجام شده است.

### تحلیل نتایج تجربی
خروجی هم‌ترازی به‌صورت زیر است:

<div dir="ltr">

| queries_in_qrels | queries_available | corpus_in_qrels | corpus_available |
|------------------|-------------------|-----------------|------------------|
| 2591 | 3237 | 3589 | 3633 |

</div>

تحلیل:
- تنها 2591 پرس‌وجو از 3237 پرس‌وجوی موجود دارای قضاوت ارتباطی هستند.
- تقریباً تمام اسناد (3589 از 3633) حداقل یک‌بار در qrels ظاهر شده‌اند.
- این عدم پوشش کامل پرس‌وجوها نشان می‌دهد که بخشی از پرس‌وجوها **فاقد ground truth** هستند و نباید در ارزیابی استفاده شوند.

</div>


In [10]:
alignment_df = _pd.DataFrame({
    "queries_in_qrels": qrels_train.df["query_id"].nunique(),
    "queries_available": query_store.df["_id"].nunique(),
    "corpus_in_qrels": qrels_train.df["corpus_id"].nunique(),
    "corpus_available": corpus_store.df["_id"].nunique()
}, index=["count"])


In [11]:
alignment_df


,queries_in_qrels,queries_available,corpus_in_qrels,corpus_available
count,2591,3237,3589,3633


<div style="text-align:center; direction:rtl; font-family:'Vazir','Tahoma',sans-serif; margin:40px 0; box-sizing: border-box;">
  <h1 style="
    font-size:26px;
    font-weight:700;
    padding:25px 20px;
    color:rgba(60, 4, 82, 0.81);
    border-radius:12px;
    background:linear-gradient(135deg,#f5f7ff,#ecebff);
    line-height:1.8;
  ">
   بخش ۲: بازیابی با استفاده از مدل‌های پیش آموزش دیده 
  </h1>
</div>


<div dir="rtl" style="
  text-align: right;
  padding: 10px;
  background-color: rgba(84, 24, 107, 0.76);
  border-radius: 10px;
  color: #ffffff;
  font-family: 'Vazir','Tahoma',sans-serif;
  margin-bottom: 10px;
  box-sizing: border-box;
">

  <h3 style="
    font-size:18px;
    font-weight: bold;
    margin-bottom:20px;
    color: #ffffff;
    text-align: right;
  ">
    الف) بازیابی بدون آموزش مدل‌ها
  </h3>

</div>



<div dir="rtl">

برای این بخش، یک سیستم رتبه‌بندی مبتنی بر الگوریتم BM25 پیاده‌سازی شد. پیاده‌سازی با استفاده از کلاس `BM25Advanced` انجام گرفت که توانایی‌های زیر را فراهم می‌کند:

1. **توکن‌سازی (Tokenization) پیشرفته:**  
   با توجه به ماهیت متون علمی و پزشکی، از یک توکنایزر سفارشی استفاده شد که قادر است:
   - کلمات مرکب و فرمول‌های علمی مانند `1H NMR` را به درستی شناسایی کند.
   - کاراکترهای جداساز مانند `,` و `/` بین کلمات انگلیسی را نادیده بگیرد و توکن‌های معنی‌دار تولید کند.
   - فقط توکن‌هایی که شامل حروف الفبا یا پرانتز هستند را نگه دارد تا اختلال ناشی از اعداد و علائم اضافی کاهش یابد.

2. **پردازش مجموعه اسناد:**  
   هر سند در مجموعه داده با:
   - استخراج `doc_id` و متن کامل (`full_text`)  
   - تولید لیست توکن‌ها با استفاده از آنالایزر سفارشی  
   - محاسبه تعداد توکن‌ها (`token_count`) و طول واقعی سند (`doc_length`)  
   آماده شد.  
   این مرحله تضمین می‌کند که ویژگی‌های مورد نیاز BM25 برای محاسبه نمره هر سند فراهم شود.




In [12]:
class BM25Advanced:
    def __init__(self, corpus_df, k1=1.5, b=0.75, ngram_range=(1,1), min_df=1):
        self.corpus_df = corpus_df.copy()
        self.k1 = k1
        self.b = b
        self.ngram_range = ngram_range
        self.vectorizer = CountVectorizer(
            lowercase=True,
            tokenizer=self._custom_tokenizer,
            ngram_range=self.ngram_range,
            min_df=min_df
        )
        self.analyzer = self.vectorizer.build_analyzer()
        self._preprocess_corpus()
        self._build_inverted_index()
        self._build_bm25_index()

    def _custom_tokenizer(self, text):
        text = re.sub(r"(?<=[A-Za-z])\,(?=[A-Za-z])", " ", text)
        
        pattern = r"""
            [A-Za-z0-9]+
            (?:[-_/][A-Za-z0-9]+)*
            (?:\([A-Za-z0-9\+]+(?:[-_/\+][A-Za-z0-9]+)*\))?
            (?:[A-Za-z0-9]+)*
            (?:\([A-Za-z0-9]+(?:[-_/\+][A-Za-z0-9]+)*\))?
        """

        tokens = re.findall(pattern, text, flags=re.VERBOSE)
        
        tokens = [t for t in tokens if re.search(r"[A-Za-z]", t) or "(" in t]
        
        return tokens

    def _preprocess_corpus(self):
        self.corpus_df["doc_id"] = self.corpus_df["_id"]
        self.corpus_df["tokens"] = self.corpus_df["full_text"].apply(self.analyzer)
        self.corpus_df["token_count"] = self.corpus_df["tokens"].apply(len)
        self.corpus_df["doc_length"] = self.corpus_df["full_text"].apply(lambda x: len(re.findall(r'\b\w+\b', x)))
        self.tokenized_corpus = self.corpus_df["tokens"].tolist()

    def _build_inverted_index(self):
        self.inverted_index = {}
        for i, tokens in enumerate(self.tokenized_corpus):
            doc_id = self.corpus_df["_id"].iloc[i]
            term_counts = Counter(tokens)
            for term, tf in term_counts.items():
                if term not in self.inverted_index:
                    self.inverted_index[term] = {"df": 0, "total_freq": 0, "posting": {}}
                self.inverted_index[term]["df"] += 1
                self.inverted_index[term]["total_freq"] += tf
                positions = [pos for pos, t in enumerate(tokens) if t == term]
                self.inverted_index[term]["posting"][doc_id] = {"tf": tf, "positions": positions}
                
    def _build_bm25_index(self):
        self.bm25 = BM25Okapi(self.tokenized_corpus, k1=self.k1, b=self.b)

    def rank_query_detailed(self, query_text, top_k=10):
        query_tokens = self.analyzer(query_text)
        scores = _np.array(self.bm25.get_scores(query_tokens))
        
        df_scores = self.corpus_df.copy()
        df_scores["score"] = scores
        df_scores = df_scores.sort_values(by=["score", "doc_id"], ascending=[False, True])
        
        results = df_scores.head(top_k).copy()
        results["rank"] = range(1, len(results)+1)
        
        results["query_tokens"] = [query_tokens]*len(results)
        overlap_terms_list = []
        tf_query_terms_list = []

        for i, row in results.iterrows():
            doc_tokens = row["tokens"]
            overlap_terms = list(set(doc_tokens) & set(query_tokens))
            tf_query_terms = {t: doc_tokens.count(t) for t in overlap_terms}
            overlap_terms_list.append(overlap_terms)
            tf_query_terms_list.append(tf_query_terms)

        results["overlap_terms"] = overlap_terms_list
        results["tf_query_terms"] = tf_query_terms_list

        return results[[
            "_id", "title", "score", "rank",
            "token_count", "doc_length",
            "query_tokens", "overlap_terms", "tf_query_terms"
        ]].reset_index(drop=True)

    def visualize_query(self, query_text, top_k=10):
        top_docs = self.rank_query(query_text, top_k)
        fig = _go.Figure()
        fig.add_trace(
            _go.Bar(
                x=top_docs["score"],
                y=top_docs["title"].apply(lambda x: x if x else "(No Title)"),
                orientation='h',
                text=top_docs["score"].round(2),
                textposition='auto'
            )
        )
        fig.update_layout(
            title=f"Top-{top_k} BM25 Scores for Query",
            xaxis_title="BM25 Score",
            yaxis_title="Document Title",
            yaxis=dict(autorange="reversed")
        )
        fig.show()
        return top_docs

    def show_doc_with_tokens(self, doc_id):
        row = self.corpus_df.loc[self.corpus_df["_id"] == doc_id].iloc[0]
        doc_text = row["full_text"]
        tokens = row["tokens"]

        print(f"Document ID: {doc_id}\n")
        print("=== Full Text ===\n")
        print(doc_text, "\n")
        print("=== Tokens ===\n")
        print(tokens)
        print(f"\nTotal tokens: {len(tokens)}")

    def inverted_index_preview_random(self, n_terms=5, max_docs=4):
        all_terms = list(self.inverted_index.keys())
        dict_rows = []
        posting_rows = []

        for term in all_terms[:n_terms]:
            info = self.inverted_index[term]

            dict_rows.append({
                "term": term,
                "df": info["df"],
                "total_freq": info["total_freq"]
            })

            docs = list(info["posting"].items())
            if info["df"] > max_docs:
                docs_to_show = docs[:max_docs]
            else:
                docs_to_show = docs

            for doc_id, doc_info in docs_to_show:
                posting_rows.append({
                    "term": term,
                    "doc_id": doc_id,
                    "tf": doc_info["tf"],
                    "positions": doc_info["positions"]
                })

            if info["df"] > max_docs:
                posting_rows.append({
                    "term": term,
                    "doc_id": "...",
                    "tf": "...",
                    "positions": "..."
                })

        df_dict_preview = _pd.DataFrame(dict_rows)
        df_posting_preview = _pd.DataFrame(posting_rows)

        return df_dict_preview, df_posting_preview


In [13]:
bm25_engine = BM25Advanced(corpus_store.df, ngram_range=(1,1))

In [14]:
docs_id = ['MED-1470', 'MED-962', 'MED-5366', 'MED-1739', 'MED-4864']

for doc_id in docs_id:
	bm25_engine.show_doc_with_tokens(doc_id)
	print('*'*80)

Document ID: MED-1470

=== Full Text ===

Intramyocellular lipid concentrations are correlated with insulin sensitivity in humans: a 1H NMR spectroscopy study. Recent muscle biopsy studies have shown a relation between intramuscular lipid content and insulin resistance. The aim of this study was to test this relation in humans by using a novel proton nuclear magnetic resonance (1H NMR) spectroscopy technique, which enables non-invasive and rapid (approximately 45 min) determination of intramyocellular lipid (IMCL) content. Normal weight non-diabetic adults (n = 23, age 29+/-2 years. BMI = 24.1+/-0.5 kg/m2) were studied using cross-sectional analysis. Insulin sensitivity was assessed by a 2-h hyperinsulinaemic (approximately 450 pmol/l)-euglycaemic (approximately 5 mmol/l) clamp test. Intramyocellular lipid concentrations were determined by using localized 1H NMR spectroscopy of soleus muscle. Simple linear regression analysis showed an inverse correlation (r = -0.579, p = 0.0037) [corr


<div dir="rtl">


3. **ایندکس معکوس (Inverted Index):**  
   ایندکس معکوس با ساختار زیر ایجاد شد:
   - `term` → `df` (تعداد اسنادی که ترم در آن‌ها ظاهر شده)  
   - `total_freq` (تعداد کل وقوع ترم در کل مجموعه)  
   - `posting` → `{doc_id: {"tf": تعداد وقوع، "positions": مکان‌های وقوع}}`  
   
   این ساختار امکان دسترسی سریع به ترم‌ها و مشاهده جزئیات هر سند شامل تعداد و موقعیت توکن‌ها را فراهم می‌کند. نمونه‌ای از ایندکس معکوس استخراج شده:

   <div dir="ltr">

   Dictionary Preview:
   | term    | df  | total_freq |
   |---------|-----|------------|
   | statin  | 16  | 65         |
   | use     | 505 | 826        |
   | and     | 3624| 33887      |
   | breast  | 267 | 1078       |
   | cancer  | 731 | 3004       |

   Posting Preview:
   | term   | doc_id   | tf | positions                     |
   |--------|----------|----|-------------------------------|
   | statin | MED-10   | 9  | [0, 51, 83, 109, 112, ...]   |
   | statin | MED-14   | 4  | [0, 128, 162, 212]            |
   | use    | MED-10   | 7  | [1, 84, 113, 158, 180, 200, 213] |
   | breast | MED-10   | 8  | [3, 32, 47, 58, 68, 142, 165, 246] |
   | cancer | MED-10   | 10 | [4, 33, 48, 59, 69, 79, 143, 166, 195, 247] |

   </div>

</div>


In [15]:
df_dict, df_posting = bm25_engine.inverted_index_preview_random(n_terms=5, max_docs=4)

print("=== Dictionary Preview ===")
df_dict


=== Dictionary Preview ===


,term,df,total_freq
0,statin,16,65
1,use,505,826
2,and,3624,33887
3,breast,267,1078
4,cancer,731,3004


In [16]:
print("\n=== Posting Preview ===")
df_posting



=== Posting Preview ===


,term,doc_id,tf,positions
0,statin,MED-10,9,"[0, 51, 83, 109, 112, 157, 179, 199, 224]"
1,statin,MED-14,4,"[0, 128, 162, 212]"
2,statin,MED-1193,9,"[7, 58, 77, 117, 120, 144, 329, 387, 403]"
3,statin,MED-1207,1,[253]
4,statin,...,...,...
5,use,MED-10,7,"[1, 84, 113, 158, 180, 200, 213]"
6,use,MED-14,2,"[1, 163]"
7,use,MED-332,1,[237]
8,use,MED-335,1,[29]
9,use,...,...,...



<div dir="rtl">

4. **ایندکس BM25 و رتبه‌بندی پرس‌وجو:**  
   با استفاده از کتابخانه `rank_bm25` و پارامترهای استاندارد BM25 (`k1=1.5, b=0.75`) ایندکس BM25 ایجاد شد. سپس برای هر پرس‌وجو:
   - توکن‌های پرس‌وجو استخراج شدند.
   - نمره BM25 هر سند محاسبه شد.
   - اسناد top-k با بیشترین نمره مرتب و همراه با جزئیات شامل تعداد توکن، طول سند، توکن‌های مشترک با پرس‌وجو و فراوانی آن‌ها بازگردانده شدند.

   نمونه‌ای از نتایج رتبه‌بندی برای پرس‌وجوی `"Why Deep Fried Foods May Cause Cancer"` با k=5:

   <div dir="ltr">

   | Rank | Doc ID   | Score   | Title                                                                 | Token Count | Doc Length | Overlap Terms             | TF Query Terms                         |
   |------|----------|---------|-----------------------------------------------------------------------|-------------|------------|---------------------------|---------------------------------------|
   | 1    | MED-2418 | 15.6858 | Consumption of deep-fried foods and risk of prostate cancera,b       | 242         | 300        | ['fried','may','foods','cancer'] | {'fried':5,'may':2,'foods':5,'cancer':1} |
   | 2    | MED-1027 | 14.2749 | Varicose Veins, Deep Vein Thrombosis, and Haemorrhoids               | 57          | 58         | ['cause','deep']           | {'cause':1,'deep':2}                  |
   | 3    | MED-2195 | 13.8726 | The influence of deep frying using various vegetable oils on acrylamide formation in sweet potato chips | 177         | 191        | ['deep','fried']           | {'deep':1,'fried':2}                  |
   | 4    | MED-4070 | 13.6785 | Intake of fried meat and risk of cancer                               | 218         | 239        | ['may','fried','cancer']   | {'may':1,'fried':6,'cancer':10}      |
   | 5    | MED-5033 | 12.9684 | Cancer is a Preventable Disease that Requires Major Lifestyle Changes | 199         | 214        | ['cause','foods','fried','cancer'] | {'cause':1,'foods':1,'fried':1,'cancer':6} |

   </div>

## تحلیل عمیق نتایج تجربی

1. **توانایی تفکیک ترم‌ها و وزن‌دهی BM25:**  
   مشاهده می‌شود که نمره BM25 با تعداد وقوع ترم‌های پرس‌وجو در سند ارتباط مستقیم دارد. به عنوان مثال، در سند MED-4070، ترم `cancer` ۱۰ بار تکرار شده و نمره بالایی برای این سند ایجاد کرده است. این همخوانی نشان می‌دهد که BM25 به طور موثر **فراوانی ترم (TF)** و **تعداد اسناد شامل ترم (DF)** را در محاسبه وزن لحاظ می‌کند.

2. **ارتباط بین طول سند و نمره:**  
   اسناد طولانی‌تر با تعداد توکن زیاد، مانند MED-2418، با وجود تکرار کمتر برخی ترم‌ها، به دلیل وجود چندین ترم مرتبط با پرس‌وجو در متن، نمره بالایی کسب کرده‌اند. این نشان‌دهنده تأثیر **پارامتر b** در تنظیم طول سند در فرمول BM25 است:

   <div align="center" dir="ltr">

   $$
   \text{score}(q,d) = \sum_{t \in q} IDF(t) \cdot \frac{tf_{t,d} \cdot (k_1 + 1)}{tf_{t,d} + k_1 \cdot (1 - b + b \cdot \frac{|d|}{avgdl})}
   $$

   </div>

   که در آن $tf_{t,d}$ تعداد وقوع ترم در سند، $|d|$ طول سند و $avgdl$ میانگین طول اسناد است.

3. **تحلیل هم‌نشینی ترم‌ها با پرس‌وجو:**  
   در نتایج استخراج شده، هم‌نشینی ترم‌های پرس‌وجو و سند به وضوح قابل مشاهده است. برای مثال، سند MED-2418 چهار ترم از پرس‌وجو را پوشش می‌دهد، در حالی که سند MED-1027 تنها دو ترم همپوشانی دارد. این امر نشان می‌دهد که BM25 علاوه بر فراوانی، **تطبیق دقیق ترم‌ها با پرس‌وجو** را نیز در رتبه‌بندی لحاظ می‌کند.

4. **ایندکس معکوس به عنوان ابزار تحلیل:**  
   ایندکس معکوس تولید شده امکان بررسی جزئیات زیر را فراهم می‌کند:
   - فراوانی هر ترم در اسناد مختلف.
   - موقعیت‌های وقوع ترم‌ها برای تحلیل‌های پیچیده‌تر مانند **co-occurrence** و **semantic proximity**.
   این اطلاعات پایه‌ای ارزشمند برای توسعه مدل‌های پیشرفته مبتنی بر روابط هم‌نشینی و تحلیل اطلاعات متقابل فراهم می‌کند.

</div>


In [17]:
top_k = 5

query_row = query_store.df.loc[query_store.df["_id"] == "PLAIN-1"].iloc[0]
query_text = query_row["text"]
query_id = query_row["_id"]

top_docs = bm25_engine.rank_query_detailed(query_text, top_k=top_k)

top_docs["_query_id"] = query_id


In [18]:
def print_query_results_detailed(query_id, query_text, df):
    query_tokens = df["query_tokens"].iloc[0] if "query_tokens" in df.columns else []
    
    print("="*100)
    print(f"QUERY_ID: {query_id}")
    print(f"QUERY: {query_text}")
    print(f"Tokens: {query_tokens}")
    print("="*100, "\n")
    
    for idx, row in df.iterrows():
        print(f"Rank {row['rank']} | Doc ID: {row['_id']} | Score: {row['score']:.4f}")
        print(f"Title: {row['title']}\n")
        print(f"Document Stats:")
        print(f"  Token Count: {row['token_count']}")
        print(f"  Doc Length: {row['doc_length']}\n")
        print(f"Overlap with Query:")
        print(f"  Terms: {row['overlap_terms']}")
        print(f"  Term Frequencies: {row['tf_query_terms']}\n")
        print("-"*100)


print_query_results_detailed(query_id, query_text, top_docs)


QUERY_ID: PLAIN-1
QUERY: Why Deep Fried Foods May Cause Cancer
Tokens: ['why', 'deep', 'fried', 'foods', 'may', 'cause', 'cancer']

Rank 1 | Doc ID: MED-2418 | Score: 15.6858
Title: Consumption of deep-fried foods and risk of prostate cancera,b

Document Stats:
  Token Count: 242
  Doc Length: 300

Overlap with Query:
  Terms: ['cancer', 'may', 'fried', 'foods']
  Term Frequencies: {'cancer': 1, 'may': 2, 'fried': 5, 'foods': 5}

----------------------------------------------------------------------------------------------------
Rank 2 | Doc ID: MED-1027 | Score: 14.2749
Title: Varicose Veins, Deep Vein Thrombosis, and Haemorrhoids: Epidemiology and Suggested Aetiology

Document Stats:
  Token Count: 57
  Doc Length: 58

Overlap with Query:
  Terms: ['deep', 'cause']
  Term Frequencies: {'deep': 2, 'cause': 1}

----------------------------------------------------------------------------------------------------
Rank 3 | Doc ID: MED-2195 | Score: 13.8726
Title: The influence of deep fryi

<div dir="rtl" style="
  text-align: right;
  padding: 10px;
  background-color: rgba(84, 24, 107, 0.76);
  border-radius: 10px;
  color: #ffffff;
  font-family: 'Vazir','Tahoma',sans-serif;
  margin-bottom: 10px;
  box-sizing: border-box;
">

  <h3 style="
    font-size:18px;
    font-weight: bold;
    margin-bottom:20px;
    color: #ffffff;
    text-align: right;
  ">
   ب) بازیابی انباشته مبتنی بر bi-encoder
  </h3>

</div>


<div dir="rtl">

در این بخش، یک سیستم بازیابی اطلاعات انباشته مبتنی بر معماری **bi-encoder** پیاده‌سازی شده است که از مدل از پیش‌آموزش‌دیده  
`sentence-transformers/msmarco-distilbert-base-tas-b` استفاده می‌کند. این مدل بر پایه‌ی معماری Transformer (نسخه‌ی DistilBERT) آموزش داده شده و هدف آن نگاشت متون (اعم از پرس‌وجو و سند) به یک فضای برداری متراکم مشترک است؛ به‌گونه‌ای که نزدیکی هندسی بردارها بیانگر نزدیکی معنایی متون باشد.

</div>


In [ ]:
import os
os.environ["HF_TOKEN"] = ""


In [ ]:
bi_encoder_model = SentenceTransformer("sentence-transformers/msmarco-distilbert-base-tas-b")


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

c:\Users\rozhin\Documents\Uni Tehran\Semester 7\IIR\HW\IIR-CA5-220701046\ir_env\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rozhin\.cache\huggingface\hub\models--sentence-transformers--msmarco-distilbert-base-tas-b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [38]:
print(torch.__version__)
print(torch.cuda.is_available())


2.10.0+cpu
False


In [39]:
import sys
print(sys.executable)


c:\Users\rozhin\Documents\Uni Tehran\Semester 7\IIR\HW\IIR-CA5-220701046\ir_env\Scripts\python.exe


<div dir="rtl">

### فرآیند Indexing (رمزگذاری اسناد)

مطابق با منطق bi-encoder، تمامی اسناد موجود در پیکره **تنها یک بار** در مرحله‌ی Indexing به بردارهای متراکم تبدیل می‌شوند. در پیاده‌سازی حاضر، متن هر سند از ستون `text` استخراج شده و با استفاده از تابع `encode` مدل، به بردارهایی با بُعد ثابت (Embedding) نگاشت می‌شود. فعال‌سازی گزینه‌ی `normalize_embeddings=True` باعث می‌شود که تمامی بردارها نُرم واحد داشته باشند؛ این موضوع محاسبه‌ی شباهت کوسینوسی را به یک ضرب داخلی ساده تبدیل می‌کند و از نظر عددی پایدارتر است.

فرآیند Indexing در خروجی اجرایی نشان می‌دهد که کل پیکره در **۱۱۴ batch** پردازش شده است که بیانگر انجام موفق و کامل رمزگذاری اسناد پیش از مرحله‌ی جستجو است. این جداسازی زمانی (Indexing در برابر Search) یکی از مزیت‌های بنیادین معماری bi-encoder محسوب می‌شود.


</div>


In [51]:
documents = corpus_store.df["text"].tolist()

doc_embeddings = bi_encoder_model.encode(
    documents,
    batch_size=32,
    show_progress_bar=True,
    convert_to_tensor=True,
    normalize_embeddings=True
)

Batches:   0%|          | 0/114 [00:00<?, ?it/s]

<div dir="rtl">


### رمزگذاری پرس‌وجو و فرآیند جستجو

در زمان جستجو، تنها متن پرس‌وجو رمزگذاری می‌شود. پرس‌وجوی مورد بررسی با شناسه‌ی `PLAIN-1` و متن:

> *Why Deep Fried Foods May Cause Cancer*

ابتدا به یک بردار متراکم نرمال‌شده نگاشت شده و سپس شباهت کوسینوسی آن با تمامی بردارهای اسناد محاسبه می‌شود. از آنجا که تمامی embeddingها از پیش نرمال شده‌اند، شباهت کوسینوسی به‌صورت زیر محاسبه می‌شود:

<div align="center" dir="ltr">

$$
\text{sim}(q, d_i) = q \cdot d_i
$$

</div>

که در آن \( q \) بردار پرس‌وجو و \( d_i \) بردار سند iاُم است. در نهایت، با استفاده از عملگر `topk`، اسنادی که بیشترین مقدار شباهت را دارند استخراج و رتبه‌بندی می‌شوند.



</div>


In [52]:
def encode_query(query_text):
    return bi_encoder_model.encode(
        query_text,
        convert_to_tensor=True,
        normalize_embeddings=True
    )


In [53]:
def bi_encoder_search_detailed(query_text, top_k=5):
    query_embedding = encode_query(query_text)

    scores = util.cos_sim(query_embedding, doc_embeddings)[0]
    top_results = torch.topk(scores, k=top_k)

    rows = []
    for rank, (score, idx) in enumerate(
        zip(top_results.values, top_results.indices), start=1
    ):
        row = corpus_store.df.iloc[idx.item()]

        rows.append({
            "_id": row["_id"],
            "title": row.get("title", ""),
            "text": row.get("text", ""),
            "score": float(score),
            "rank": rank,
        })

    return _pd.DataFrame(rows)


<div dir="rtl">


## تحلیل عمیق نتایج تجربی

نتایج بازیابی برای پرس‌وجوی مورد نظر نشان می‌دهد که سیستم توانسته است اسنادی را بازیابی کند که از نظر **معنایی** ارتباط نزدیکی با مفهوم «غذاهای سرخ‌شده و خطر سرطان» دارند، حتی در مواردی که هم‌پوشانی واژگانی مستقیم وجود ندارد.

### تفسیر رتبه‌های برتر

- **رتبه‌ی ۱ (MED-2418 | Score = 0.8047)**  
  این سند به‌صورت مستقیم به مصرف غذاهای سرخ‌شده و افزایش ریسک سرطان (به‌طور خاص سرطان پروستات) اشاره می‌کند و حتی مکانیسم‌های شیمیایی (مانند تشکیل ترکیبات بالقوه سرطان‌زا) را توضیح می‌دهد. بالاترین امتیاز شباهت کوسینوسی نشان می‌دهد که embedding این سند در فضای برداری بیشترین نزدیکی را به embedding پرس‌وجو دارد؛ این نتیجه با انتظار معنایی کاملاً سازگار است.

- **رتبه‌ی ۲ (MED-4070 | Score = 0.8002)**  
  تمرکز این سند بر گوشت سرخ‌شده و ارتباط آن با انواع سرطان است. با وجود تفاوت جزئی در نوع ماده‌ی غذایی (گوشت در برابر غذاهای سرخ‌شده به‌طور کلی)، مدل توانسته است رابطه‌ی معنایی «سرخ‌کردن → ترکیبات جهش‌زا → سرطان» را به‌خوبی پیدا کند. این امر نشان‌دهنده‌ی توان مدل در درک روابط هم‌نشینی معنایی فراتر از تطابق واژگانی است.

- **رتبه‌های ۳ تا ۵**  
  اسناد این رتبه‌ها بیشتر به عوامل سبک زندگی، رژیم غذایی و نقش آن‌ها در بروز سرطان می‌پردازند. اگرچه تمرکز آن‌ها به‌صورت مستقیم روی «deep fried foods» نیست، اما از منظر معنایی در همان خوشه‌ی مفهومی «رژیم غذایی و سرطان» قرار می‌گیرند. کاهش تدریجی امتیاز شباهت (از حدود 0.79 به پایین) به‌خوبی نشان‌دهنده‌ی فاصله‌ی معنایی بیشتر نسبت به اسناد رتبه‌های بالاتر است.

این توزیع امتیازها بیانگر آن است که مدل bi-encoder توانسته است یک **رتبه‌بندی پیوسته و معنادار** بر اساس نزدیکی معنایی ایجاد کند، بدون آنکه صرفاً به تکرار کلمات کلیدی وابسته باشد.

</div>


In [54]:
query_row = query_store.df.loc[query_store.df["_id"] == "PLAIN-1"].iloc[0]
query_text = query_row["text"]

top_docs = bi_encoder_search_detailed(
    query_text=query_text,
    top_k=5
)


In [55]:
def print_bi_encoder_query_results_detailed(query_id, query_text, df):
    print("="*100)
    print(f"QUERY_ID: {query_id}")
    print(f"QUERY: {query_text}")
    print("="*100, "\n")
    
    for idx, row in df.iterrows():
        print(f"Rank {row['rank']} | Doc ID: {row['_id']} | Score: {row['score']:.4f}")
        print(f"Title: {row['title']}\n")
        print(f"Text: {row['text']}")
        print("-"*100)


print_bi_encoder_query_results_detailed(query_id, query_text, top_docs)


QUERY_ID: PLAIN-1
QUERY: Why Deep Fried Foods May Cause Cancer

Rank 1 | Doc ID: MED-2418 | Score: 0.8047
Title: Consumption of deep-fried foods and risk of prostate cancera,b

Text: Background Evidence suggests that high-heat cooking methods may increase the risk of prostate cancer (PCa). The addition of oil/fat, as in deep-frying, may be of particular concern, and has not specifically been investigated in relation to PCa. Potential mechanisms include the formation of potentially carcinogenic agents such as aldehydes, acrolein, heterocyclic amines, polycyclic aromatic hydrocarbons, and acrylamide. Methods We estimated odds ratios (OR) and 95% confidence intervals (CI) for the association between tertiles of intake of deep-fried foods from a food frequency questionnaire (French fries, fried chicken, fried fish, doughnuts and snack chips) and PCa risk, adjusted for potential confounders, among 1,549 cases and 1,492 controls. We additionally examined associations with more aggressive PCa

<div dir="rtl" style="
  text-align: right;
  padding: 30px;
  background-color: rgba(60, 4, 82, 0.21);
  border-radius: 14px;
  color: #ffffff;
  font-family: 'Vazir','Tahoma',sans-serif;
  margin-bottom: 30px;
  box-shadow: 0 8px 16px rgba(60, 4, 82, 0.21);
  box-sizing: border-box;
">

  <p style="font-size:16px; line-height:2; margin-bottom:20px; text-align: justify;">
    تفاوت بنیادی معماری بین bi-encoder و cross-encoder را توضیح دهید. چرا bi-encoder می‌توانند document embeddings را پیش‌محاسبه کنند اما cross-encoder نمی‌توانند؟ پیچیدگی محاسباتی بازیابی از بین N سند برای هر رویکرد چیست؟
  </p>

  <div style="
    padding: 20px;
    background-color: #d7c5e7ff;
    border-radius: 12px;
    border: 2px solid rgba(60, 4, 82, 0.21);
    color: #080808ff;
    margin-bottom: 15px;
    box-sizing: border-box;
  ">

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	<b>1) تفاوت معماری Bi-Encoder در برابر Cross-Encoder</b>
</p>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	تفاوت بنیادی میان معماری‌های bi-encoder و cross-encoder در نحوه‌ی تعامل بین پرس‌وجو و سند نهفته است. این تفاوت مستقیماً بر امکان پیش‌محاسبه‌ی نمایش برداری اسناد و همچنین پیچیدگی محاسباتی فرآیند بازیابی اثر می‌گذارد.
</p>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	در معماری <b>bi-encoder</b>، پرس‌وجو و سند به‌صورت کاملاً مستقل از یکدیگر رمزگذاری می‌شوند. هر متن (چه پرس‌وجو و چه سند) به‌طور جداگانه وارد مدل شده و به یک بردار متراکم در فضای معنایی مشترک نگاشت می‌شود. در این ساختار، هیچ‌گونه سازوکار attention مستقیمی میان توکن‌های پرس‌وجو و توکن‌های سند وجود ندارد. به همین دلیل، بردارهای اسناد می‌توانند یک‌بار در مرحله‌ی Indexing محاسبه و ذخیره شوند و در زمان جستجو، تنها بردار پرس‌وجو محاسبه شده و شباهت آن با بردارهای از پیش محاسبه‌شده‌ی اسناد سنجیده می‌شود.
</p>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	در مقابل، در معماری <b>cross-encoder</b>، پرس‌وجو و سند به‌صورت یک توالی واحد 
	(مانند <code> [CLS] query [SEP] document [SEP] </code> )
	 به مدل داده می‌شوند. در این حالت، مکانیزم attention بین تمامی توکن‌های پرس‌وجو و سند برقرار می‌شود و مدل می‌تواند برهم‌کنش‌های ظریف و دقیق بین آن‌ها را مستقیماً مدل‌سازی کند. با این حال، امتیاز شباهت تنها با پردازش هم‌زمان هر جفت (پرس‌وجو، سند) قابل محاسبه است؛ بنابراین امکان پیش‌محاسبه و ذخیره‌ی embedding اسناد وجود ندارد.
</p>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	<b>2) پیچیدگی محاسباتی بازیابی</b>
</p>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	اگر تعداد اسناد موجود در پیکره را با <i>N</i> و بُعد بردارهای متراکم را با <i>d</i> نشان دهیم، در رویکرد bi-encoder، در زمان جستجو تنها یک embedding برای پرس‌وجو محاسبه شده و سپس شباهت آن با تمامی <i>N</i> سند از طریق ضرب داخلی یا شباهت کوسینوسی محاسبه می‌شود. در نتیجه، پیچیدگی زمانی این فرآیند از مرتبه‌ی زیر است:
</p>

<div align="center" dir="ltr" style="margin-bottom:15px;">

$$
\mathcal{O}(N \cdot d)
$$

</div>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	در مقابل، در رویکرد cross-encoder، مدل باید برای هر پرس‌وجو، <i>N</i> بار اجرا شود (یک بار برای هر سند)، که منجر به پیچیدگی زمانی بسیار بالاتری می‌شود:
</p>

<div align="center" dir="ltr" style="margin-bottom:15px;">

$$
\mathcal{O}(N \cdot \text{Transformer})
$$

</div>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	این تفاوت بنیادین سبب می‌شود که bi-encoderها برای بازیابی اولیه (Retrieval) در مقیاس‌های بزرگ بسیار مناسب‌تر باشند، در حالی که cross-encoderها معمولاً در مرحله‌ی بازرتبه‌بندی (Re-ranking) و بر روی مجموعه‌ای محدود از اسناد به کار گرفته می‌شوند.
</p>

</div>


<div dir="rtl" style="
  text-align: right;
  padding: 10px;
  background-color: rgba(84, 24, 107, 0.76);
  border-radius: 10px;
  color: #ffffff;
  font-family: 'Vazir','Tahoma',sans-serif;
  margin-bottom: 10px;
  box-sizing: border-box;
">

  <h3 style="
    font-size:18px;
    font-weight: bold;
    margin-bottom:20px;
    color: #ffffff;
    text-align: right;
  ">
   پ) بازیابی متراکم مبتنی بر پایه cross-encoder
  </h3>

</div>


<div dir="rtl">

در این قسمت، برای بازیابی متون مرتبط با پرس‌وجوی `"PLAIN-1"` که موضوع آن `"Why Deep Fried Foods May Cause Cancer"` است، از یک خط لوله دو مرحله‌ای مبتنی بر Bi-Encoder و Cross-Encoder استفاده شد. این رویکرد به منظور ترکیب مزایای **کارایی بالا** و **دقت بالا** در بازیابی اطلاعات طراحی شده است.

### مرحله اول: بازیابی اولیه با Bi-Encoder

در ابتدا، مدل Bi-Encoder برای استخراج ۱۰۰ سند کاندید از مجموعه متون مورد استفاده قرار گرفت. این مرحله با فراخوانی تابع `bi_encoder_search_detailed(query_text, top_k=100)` انجام شد. خروجی این مرحله شامل رتبه‌بندی اولیه بر اساس امتیاز **Bi Score** بود که نشان‌دهنده شباهت برداری بین بردار پرس‌وجو و بردار سند است. جدول زیر نمونه‌ای از ۱۰ نتیجه برتر Bi-Encoder را نشان می‌دهد:

<div align="center" dir="ltr">

| Rank | Doc ID   | Bi Score | Title |
|------|----------|----------|-------|
| 1    | MED-2418 | 0.8047   | Consumption of deep-fried foods and risk of prostate cancera,b |
| 2    | MED-4070 | 0.8002   | Intake of fried meat and risk of cancer: a follow-up study in Finland. |
| 3    | MED-4391 | 0.7928   | Lifestyle related causes of cancer and chemoprevention through phytonutrients. |
| 4    | MED-5033 | 0.7921   | Cancer is a Preventable Disease that Requires Major Lifestyle Changes |
| 5    | MED-3000 | 0.7884   | Red meat consumption and cancer: reasons to suspect involvement of bovine infectious factors in colorectal cancer. |
| 6    | MED-1151 | 0.7845   | Organic food consumption and the incidence of cancer in a large prospective study of women in the United Kingdom |
| 7    | MED-1817 | 0.7759   | Meat and fish consumption and risk of pancreatic cancer: results from the European Prospective Investigation into Cancer and Nutrition. |
| 8    | MED-1540 | 0.7754   | Vegetarian diets: what do we know of their effects on common chronic diseases? |
| 9    | MED-2439 | 0.7705   | Plant Sterols as Anticancer Nutrients: Evidence for Their Role in Breast Cancer |
| 10   | MED-2423 | 0.7696   | Dietary patterns and breast cancer risk among women. |

</div>

</div>


In [127]:
query_row = query_store.df.loc[query_store.df["_id"] == "PLAIN-1"].iloc[0]
query_text = query_row["text"]

top_100_docs_Bi = bi_encoder_search_detailed(
    query_text=query_text,
    top_k=100
)


In [128]:
print_bi_encoder_query_results_detailed(query_id, query_text, top_100_docs_Bi.head(10))
print('....')

QUERY_ID: PLAIN-1
QUERY: Why Deep Fried Foods May Cause Cancer

Rank 1 | Doc ID: MED-2418 | Score: 0.8047
Title: Consumption of deep-fried foods and risk of prostate cancera,b

Text: Background Evidence suggests that high-heat cooking methods may increase the risk of prostate cancer (PCa). The addition of oil/fat, as in deep-frying, may be of particular concern, and has not specifically been investigated in relation to PCa. Potential mechanisms include the formation of potentially carcinogenic agents such as aldehydes, acrolein, heterocyclic amines, polycyclic aromatic hydrocarbons, and acrylamide. Methods We estimated odds ratios (OR) and 95% confidence intervals (CI) for the association between tertiles of intake of deep-fried foods from a food frequency questionnaire (French fries, fried chicken, fried fish, doughnuts and snack chips) and PCa risk, adjusted for potential confounders, among 1,549 cases and 1,492 controls. We additionally examined associations with more aggressive PCa

<div dir="rtl" >

### مرحله دوم: بازرتبه‌بندی با Cross-Encoder

در مرحله دوم، همان ۱۰۰ سند کاندید توسط مدل Cross-Encoder `cross-encoder/ms-marco-MiniLM-L-6-v2` دوباره رتبه‌بندی شد. برای هر جفت پرس‌وجو-سند، Cross-Encoder یک امتیاز **Cross Score** محاسبه کرد که بازتاب‌دهنده **شباهت عمیق و دقیق متنی** بین پرس‌وجو و متن سند است. سپس اسناد بر اساس Cross Score نزولی مرتب شدند و رتبه جدید (`cross_rank`) محاسبه شد.

تابع `cross_encoder_rerank` این فرآیند را به صورت خودکار انجام داده و خروجی مرحله دوم شامل ۱۰ سند برتر بر اساس امتیاز Cross-Encoder است. جدول ترکیبی Bi-Encoder و Cross-Encoder به همراه تغییر رتبه (Change) در زیر آمده است:

<div align="center" dir="ltr">

| Rank | Bi Doc ID | Bi Score | Change | Cross Doc ID | Cross Score |
|------|-----------|----------|--------|--------------|-------------|
| 1    | MED-2418  | 0.8047   | 0 ➡️  | MED-2418     | 5.7222      |
| 2    | MED-4070  | 0.8002   | 0 ➡️  | MED-4070     | 4.3992      |
| 3    | MED-4391  | 0.7928   | -11 ⬇️ | MED-4493     | 0.2416      |
| 4    | MED-5033  | 0.7921   | 0 ➡️  | MED-5033     | 0.2064      |
| 5    | MED-3000  | 0.7884   | -4 ⬇️  | MED-5269     | 0.0601      |
| 6    | MED-1151  | 0.7845   | -4 ⬇️  | MED-3553     | -1.3581     |
| 7    | MED-1817  | 0.7759   | -8 ⬇️  | MED-3782     | -1.9202     |
| 8    | MED-1540  | 0.7754   | -36 ⬇️ | MED-3245     | -2.0420     |
| 9    | MED-2439  | 0.7705   | -19 ⬇️ | MED-3000     | -2.9570     |
| 10   | MED-2423  | 0.7696   | -37 ⬇️ | MED-1151     | -3.1434     |

</div>

### تحلیل نتایج 

1. **پایداری رتبه‌ها:** همان‌طور که مشاهده می‌شود، اسناد MED-2418 و MED-4070 در هر دو مدل Bi-Encoder و Cross-Encoder در رتبه‌های ۱ و ۲ باقی مانده‌اند. این نشان‌دهنده همگرایی قوی بین **نمایش برداری ساده** و **رتبه‌بندی دقیق محتوایی** است.

2. **تغییرات قابل توجه:** برخی اسناد مانند MED-2423 و MED-1540 در Cross-Encoder به شدت کاهش رتبه داشته‌اند (`-37` و `-36`). این امر نشان می‌دهد که Bi-Encoder ممکن است شباهت سطحی یا هم‌نشینی سطحی (Syntagmatic) را ثبت کرده باشد، اما Cross-Encoder پس از تحلیل دقیق معنایی، ارتباط واقعی را کمتر ارزیابی کرده است.

3. **دقت بالاتر Cross-Encoder:** Cross Score برای اسناد برتر (مثلاً MED-2418 = 5.7222) به وضوح از Bi Score بالاتر و متفاوت است. این نشان‌دهنده قدرت Cross-Encoder در **تحلیل روابط دقیق و استخراج اطلاعات متقابل بین پرس‌وجو و سند** است.


In [60]:
cross_encoder = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

c:\Users\rozhin\Documents\Uni Tehran\Semester 7\IIR\HW\IIR-CA5-220701046\ir_env\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rozhin\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: cross-encoder/ms-marco-MiniLM-L-6-v2
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [129]:
def cross_encoder_rerank(query_text, candidates_df, top_k=10):
    pairs = [
        [query_text, doc_text]
        for doc_text in candidates_df["text"].tolist()
    ]

    scores = cross_encoder.predict(pairs)

    candidates_df = candidates_df.copy()
    candidates_df["cross_score"] = scores.astype(float)

    candidates_df = candidates_df.sort_values(
        by="cross_score",
        ascending=False
    ).reset_index(drop=True)

    candidates_df["cross_rank"] = candidates_df.index + 1

    return candidates_df.head(top_k)


In [130]:
query_row = query_store.df.loc[query_store.df["_id"] == "PLAIN-1"].iloc[0]
query_text = query_row["text"]

top_100_docs_Bi_Cross = cross_encoder_rerank(
    query_text,
    top_100_docs_Bi,
    top_k=100
)

In [131]:
top_k = 10

def print_cross_encoder_query_results_detailed(query_id, query_text, df, top_k = 10):
    print("="*100)
    print(f"QUERY_ID: {query_id}")
    print(f"QUERY: {query_text}")
    print("="*100, "\n")
    
    for idx, row in df.head(top_k).iterrows():
        print(f"Rank {row['cross_rank']} | Rank {row['rank']} | Doc ID: {row['_id']}")
        print(f"Bi_encoder Score: {row['score']:.4f} | Cross_encoder Score: {row['cross_score']:.4f}")
        print(f"Title: {row['title']}\n")
        print(f"Text: {row['text']}")
        print("-"*100)


print_cross_encoder_query_results_detailed(query_id, query_text, top_100_docs_Bi_Cross, top_k)


QUERY_ID: PLAIN-1
QUERY: Why Deep Fried Foods May Cause Cancer

Rank 1 | Rank 1 | Doc ID: MED-2418
Bi_encoder Score: 0.8047 | Cross_encoder Score: 5.7222
Title: Consumption of deep-fried foods and risk of prostate cancera,b

Text: Background Evidence suggests that high-heat cooking methods may increase the risk of prostate cancer (PCa). The addition of oil/fat, as in deep-frying, may be of particular concern, and has not specifically been investigated in relation to PCa. Potential mechanisms include the formation of potentially carcinogenic agents such as aldehydes, acrolein, heterocyclic amines, polycyclic aromatic hydrocarbons, and acrylamide. Methods We estimated odds ratios (OR) and 95% confidence intervals (CI) for the association between tertiles of intake of deep-fried foods from a food frequency questionnaire (French fries, fried chicken, fried fish, doughnuts and snack chips) and PCa risk, adjusted for potential confounders, among 1,549 cases and 1,492 controls. We additionall

In [146]:
top_k = 10
bi_df = top_100_docs_Bi.head(top_k).copy()
cross_df = top_100_docs_Bi_Cross.copy().set_index("_id")

relevant_docs = set(
    qrels_dev.df.loc[
        (qrels_dev.df["query_id"] == "PLAIN-1"),
        "corpus_id"
    ].tolist()
)

bi_df["Change"] = ""
for idx, row in bi_df.iterrows():
    doc_id = row["_id"]
    bi_rank = row["rank"]
    
    if doc_id in cross_df.index:
        cross_rank = cross_df.loc[doc_id, "cross_rank"]
        rank_change = bi_rank - cross_rank
        if cross_rank < bi_rank:
            arrow = "⬆️"
        elif cross_rank > bi_rank:
            arrow = "⬇️"
        else:
            arrow = "➡️"
        bi_df.at[idx, "Change"] = f"{rank_change} {arrow}"

combined_df = _pd.DataFrame({
    "Rank": range(1, top_k+1),
    "Bi Doc ID": bi_df["_id"],
    "Bi Score": bi_df["score"],
    "Change": bi_df["Change"],
    "Cross Doc ID": cross_df.head(top_k).reset_index()["_id"],
    "Cross Score": cross_df.head(top_k).reset_index()["cross_score"]
})

def highlight_relevant_cols(s):
    return ['background-color: darkgreen' if v in relevant_docs else '' for v in s]

styled_combined_df = combined_df.style.format({
    "Bi Score": "{:.4f}",
    "Cross Score": "{:.4f}"
}).apply(highlight_relevant_cols, subset=["Bi Doc ID", "Cross Doc ID"])

styled_combined_df


,Rank,Bi Doc ID,Bi Score,Change,Cross Doc ID,Cross Score
0,1,MED-2418,0.8047,0 ➡️,MED-2418,5.7222
1,2,MED-4070,0.8002,0 ➡️,MED-4070,4.3992
2,3,MED-4391,0.7928,-11 ⬇️,MED-4493,0.2416
3,4,MED-5033,0.7921,0 ➡️,MED-5033,0.2064
4,5,MED-3000,0.7884,-4 ⬇️,MED-5269,0.0601
5,6,MED-1151,0.7845,-4 ⬇️,MED-3553,-1.3581
6,7,MED-1817,0.7759,-8 ⬇️,MED-3782,-1.9202
7,8,MED-1540,0.7754,-36 ⬇️,MED-3245,-2.0420
8,9,MED-2439,0.7705,-19 ⬇️,MED-3000,-2.9570
9,10,MED-2423,0.7696,-37 ⬇️,MED-1151,-3.1434


<div dir="rtl" >

### مزایای خط لوله دو مرحله‌ای

استفاده از Cross-Encoder تنها برای کل مجموعه متون با حجم بالا عملی نیست، زیرا:

- **محاسباتی سنگین:** Cross-Encoder نیازمند پردازش هر جفت پرس‌وجو-سند است و محاسبه همه‌ی امتیازها برای میلیون‌ها سند بسیار زمان‌بر است.
- **کارایی پایین:** حتی با GPU، Cross-Encoder تنها برای مجموعه‌های محدود (چند هزار سند) مناسب است.

بنابراین، خط لوله دو مرحله‌ای به صورت زیر منطقی است:

1. **Bi-Encoder** → کاهش فضای جستجو به ۱۰۰ کاندید برتر با سرعت بالا (کم‌هزینه و موازی‌پذیر).  
2. **Cross-Encoder** → رتبه‌بندی دقیق و استخراج بهترین نتایج با دقت بالاتر.

این ترکیب موجب می‌شود که هم **کارایی محاسباتی حفظ شود** و هم **دقت نهایی رتبه‌بندی افزایش یابد**.

<div align="center" dir="ltr">

$$
\text{Final Ranking} = \text{CrossEncoder}\Big(\text{Top-K BiEncoder Candidates}\Big)
$$

</div>

### جمع‌بندی

خط لوله دو مرحله‌ای Bi-Encoder و Cross-Encoder، یک روش بهینه برای **توازن بین سرعت و دقت بازیابی اطلاعات متنی** است. نتایج تجربی نشان می‌دهد که اسناد با بالاترین ارتباط معنایی با پرس‌وجو توسط Cross-Encoder به خوبی شناسایی شده‌اند، در حالی که Bi-Encoder برای فیلتر سریع و کاهش فضای جستجو نقش کلیدی ایفا کرده است. تغییرات رتبه‌ها نیز اطلاعات ارزشمندی در مورد تفاوت بین **شباهت برداری** و **رتبه‌بندی معنایی دقیق** ارائه می‌کند.

</div>


<div dir="rtl" style="
  text-align: right;
  padding: 30px;
  background-color: rgba(60, 4, 82, 0.21);
  border-radius: 14px;
  color: #ffffff;
  font-family: 'Vazir','Tahoma',sans-serif;
  margin-bottom: 30px;
  box-shadow: 0 8px 16px rgba(60, 4, 82, 0.21);
  box-sizing: border-box;
">

  <p style="font-size:16px; line-height:2; margin-bottom:20px; text-align: justify;">
  چرا معمولاً خط لوله دو مرحله ای (bi-encoder + cross-encoder) استفاده می‌شود و از cross-encoder تنها استفاده نمی شود؟ از هر دو جنبه دقت و کارایی بررسی کنید.
  </p>

  <div style="
    padding: 20px;
    background-color: #d7c5e7ff;
    border-radius: 12px;
    border: 2px solid rgba(60, 4, 82, 0.21);
    color: #080808ff;
    margin-bottom: 15px;
    box-sizing: border-box;
  ">

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	<b>1) از منظر کارایی و مقیاس‌پذیری</b>
</p>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	استفاده مستقیم از cross-encoder برای تمام اسناد یک مجموعه‌ی بزرگ از لحاظ محاسباتی غیرعملی است. همان‌طور که در نتایج اجرای کد های بالا مشاهده شد، حتی ۱۰۰ سند نیز نیازمند محاسبه‌ی امتیاز جداگانه برای هر جفت پرس‌وجو-سند است. اگر تعداد اسناد <i>N</i> بسیار زیاد باشد، پیچیدگی زمانی برابر است با <div align="center" dir="ltr">
  
  $$
  \mathcal{O}(N \cdot \text{Transformer})
  $$ 

  </div> که برای مقیاس‌های میلیون‌ها سند غیرقابل قبول است.
</p>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	در مقابل، bi-encoder امکان پیش‌محاسبه embedding اسناد را فراهم می‌کند و جستجو تنها شامل محاسبه embedding پرس‌وجو و اندازه‌گیری شباهت با <i>N</i> بردار ذخیره شده است، که پیچیدگی زمانی آن برابر است با <div align="center" dir="ltr"> 

  $$
  \mathcal{O}(N \cdot d)
  $$

  </div> و به‌طور قابل توجهی سریع‌تر است. بنابراین، مرحله اول بازیابی با bi-encoder به عنوان یک فیلتر سریع برای کاهش تعداد کاندیداها عمل می‌کند.
</p>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	این فیلتر اولیه امکان می‌دهد تا cross-encoder در مرحله‌ی دوم تنها بر روی تعداد محدودی سند (مثلاً ۱۰۰ سند برتر) اجرا شود و بدین ترتیب تعادل بهینه‌ای بین کارایی و دقت برقرار شود.
</p>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	<b>2) از منظر دقت</b>
</p>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
 cross-encoder به دلیل وجود attention کامل میان توکن‌های پرس‌وجو و سند، توانایی مدل‌سازی روابط معنایی پیچیده را دارد. همان‌طور که در جدول خروجی بازرتبه‌بندی مشاهده می‌شود، جایگاه برخی اسناد با Cross-Encoder بهبود یافته است؛ برای مثال سند MED-4493 با Bi-Encoder در رتبه ۳ قرار داشت اما Cross-Encoder آن را به رتبه ۱۸ منتقل کرده است که نشان‌دهنده بازشناسی دقیق‌تر تناسب معنا و پرس‌وجو است.
</p>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	بنابراین، استفاده از cross-encoder تنها بر روی تعداد محدودی از اسناد، امکان بهره‌گیری از دقت بالای آن را بدون قربانی کردن کارایی فراهم می‌کند و به همین دلیل خط لوله دو مرحله‌ای به طور گسترده در بازیابی متنی مدرن پذیرفته شده است.
</p>

<p style="font-size:15px; line-height:1.9; margin-bottom:15px; text-align: justify;">
	نتیجه این رویکرد ترکیبی، دستیابی به دقت بالاتر در رتبه‌بندی نهایی همراه با حفظ زمان پاسخ سریع برای بازیابی از مجموعه‌های بزرگ است.
</p>

</div>
</div>


<div style="text-align:center; direction:rtl; font-family:'Vazir','Tahoma',sans-serif; margin:40px 0; box-sizing: border-box;">
  <h1 style="
    font-size:26px;
    font-weight:700;
    padding:25px 20px;
    color:rgba(60, 4, 82, 0.81);
    border-radius:12px;
    background:linear-gradient(135deg,#f5f7ff,#ecebff);
    line-height:1.8;
  ">  
  بخش ۳:  بازیابی با استفاده از مدل‌های تنظیم دقیق شده
  </h1>
</div>


<div dir="rtl" style="
  text-align: right;
  padding: 10px;
  background-color: rgba(84, 24, 107, 0.76);
  border-radius: 10px;
  color: #ffffff;
  font-family: 'Vazir','Tahoma',sans-serif;
  margin-bottom: 10px;
  box-sizing: border-box;
">

  <h3 style="
    font-size:18px;
    font-weight: bold;
    margin-bottom:20px;
    color: #ffffff;
    text-align: right;
  ">
  الف) آماده سازی داده ها
  </h3>

</div>


<div dir="rtl" style="
  text-align: right;
  padding: 10px;
  background-color: rgba(84, 24, 107, 0.76);
  border-radius: 10px;
  color: #ffffff;
  font-family: 'Vazir','Tahoma',sans-serif;
  margin-bottom: 10px;
  box-sizing: border-box;
">

  <h3 style="
    font-size:18px;
    font-weight: bold;
    margin-bottom:20px;
    color: #ffffff;
    text-align: right;
  ">
  ب) فرآیند آموزش
  </h3>

</div>


<div dir="rtl" style="
  text-align: right;
  padding: 10px;
  background-color: rgba(84, 24, 107, 0.76);
  border-radius: 10px;
  color: #ffffff;
  font-family: 'Vazir','Tahoma',sans-serif;
  margin-bottom: 10px;
  box-sizing: border-box;
">

  <h3 style="
    font-size:18px;
    font-weight: bold;
    margin-bottom:20px;
    color: #ffffff;
    text-align: right;
  ">
  د) سوال نظری: مفهوم زیر را توضیح دهید:
  </h3>

</div>


<div style="text-align:center; direction:rtl; font-family:'Vazir','Tahoma',sans-serif; margin:40px 0; box-sizing: border-box;">
  <h1 style="
    font-size:26px;
    font-weight:700;
    padding:25px 20px;
    color:rgba(60, 4, 82, 0.81);
    border-radius:12px;
    background:linear-gradient(135deg,#f5f7ff,#ecebff);
    line-height:1.8;
  ">  
  بخش ۴: آموزش مدل‌ها با استفاده از تنظیم دقیق کارآمد پارامتری
  </h1>
</div>


<div style="text-align:center; direction:rtl; font-family:'Vazir','Tahoma',sans-serif; margin:40px 0; box-sizing: border-box;">
  <h1 style="
    font-size:26px;
    font-weight:700;
    padding:25px 20px;
    color:rgba(60, 4, 82, 0.81);
    border-radius:12px;
    background:linear-gradient(135deg,#f5f7ff,#ecebff);
    line-height:1.8;
  ">  
  بخش ۵: ارزیابی جامع مدل‌‌ها
  </h1>
</div>


<div style="text-align:center; direction:rtl; font-family:'Vazir','Tahoma',sans-serif; margin:40px 0; box-sizing: border-box;">
  <h1 style="
    font-size:26px;
    font-weight:700;
    padding:25px 20px;
    color:rgba(60, 4, 82, 0.81);
    border-radius:12px;
    background:linear-gradient(135deg,#f5f7ff,#ecebff);
    line-height:1.8;
  ">  
  بخش ۶: رنگ کردن HYBRID
  </h1>
</div>


<div style="text-align:center; direction:rtl; font-family:'Vazir','Tahoma',sans-serif; margin:40px 0; box-sizing: border-box;">
  <h1 style="
    font-size:22px;
    font-weight:700;
    padding:25px 20px;
    color:rgba(60, 4, 82, 0.81);
    border-radius:12px;
    background:linear-gradient(135deg,#f5f7ff,#ecebff);
    line-height:1.8;
  ">  
  بخش ۷: پیاده سازی یک RERANKER مبتنی بر LLM با استفاده از GPT-5.1-MINI API
  </h1>
</div>
